[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/LSTM/210_final_project_LSTM.ipynb)

In [0]:
# Import Libs 
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math
#from  scipy.ndimage import convolve1d

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Keras try model 
def try_model(model, data, name, epochs=200, verbose=1):  
  model.compile(optimizer='nadam',
            loss='categorical_crossentropy',
            metrics=[ 'accuracy'])

  model.summary()  
  history = model.fit(data['x_train'], data['y_train'], 
            epochs=epochs, batch_size=64, verbose=verbose, 
            validation_data=(data['x_val'], data['y_val']))  
  score = model.evaluate(data['x_test'], data['y_test'])
  print('Test accuracy: {}'.format(score))
  
#   out_path = 'serialized_models/{}.h5'.format(name)  
#   model.save(out_path)
  
#   with open('serialized_models/{}_history.pickle'.format(name), 'wb') as handle:
#     pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return history

In [0]:
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size,trials):
  """
  m - highest integer for the randint
  """
  y = np.random.randint(m,size=(trials,size))
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  #x = a + 1.*1j*b
  # lets hold x in a dp array([real,imag])
  #x = np.array([a,1.*b])
  x = np.dstack((a,b))
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  
  [x,y] = qammod2(m,sequenceLen,trials)
  
  h = [1., 0.5, 0.3, -0.13, 0.2, 0, 0.4]
  #h = np.random.randn(1,chanelLen)
  #h = [1]
  #print(x.shape)
#   xh = np.empty((sequenceLen+len(h)-1,2))
#   xh[:,0] = np.convolve(x[:,0],h)   
#   xh[:,1] = np.convolve(x[:,1],h)   
  #xh = convolve1d(x,h,axis = 1)
  xh = np.apply_along_axis(lambda m: np.convolve(m,h[0:chanelLen],mode="full"),axis = 1,arr=x)
  #print(xh.shape)
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=xh.shape) 
    xh = xh + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xh ,y ]
  
                        
data={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)

In [0]:
# Visulizing the data

m = 4         # order of modulation
#seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 10000   # number of trials 
Noise = 1
SNRdB = 20
chanelLen = 4


x = 4
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two  
print(seqLen)

generateData2(m,seqLen,trials,Noise,SNRdB,chanelLen)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels
#print(data['x_train']) # so u can see the data 

# tmp = (data['x_train'][:,:-1].reshape(trials,seqLen))

# #plot 1
# plt.scatter(np.real(tmp[0][:]),np.imag(tmp[0][:]))   
#                                          # I only plot the first 10 points from 
#                                          # the sequence because as time goes on 
#                                          # the data becomes less dependent on the
#                                          # previous symbols. Even 10 is a lot. 
# plt.title('4-QAM Mod With Multi-Channel AWGN plot 1')
# plt.show()

#plot 2
# Just a pure 16 qam with 1 point on each constalation point 
# clearly we have 16 classes
# [xtmp,ytmp]=  qammod(16,data['y_train'])
# plt.scatter(xtmp[:,0],xtmp[:,1])
# plt.title('16-QAM Mod plot 2')
# plt.show()



256
(10000, 259, 2)
(10000, 256)


In [0]:

# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

  
y_train = keras.utils.to_categorical(data['y_train'] , num_classes=m)
y_val = keras.utils.to_categorical(data['y_val'] , num_classes=m)
y_test = keras.utils.to_categorical(data['y_test'] , num_classes=m)

print(data['x_train'].shape)
print(data['x_test'].shape)
print(y_train.shape)
print(data['x_val'].shape)


default_data = {
  'x_train': data['x_train'],
  'x_val': data['x_val'],
  'x_test': data['x_test'],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}


(10000, 259, 2)
(10000, 259, 2)
(10000, 256, 4)
(5000, 259, 2)


In [0]:
# QAM-4
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(2,kernel_size=(3), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(2,kernel_size=(3)))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,bias_regularizer=regularizers.l2(0.005)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(BatchNormalization())
#modCNN.add(Dropout(0.4))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=3)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_51 (Conv1D)           (None, 257, 2)            14        
_________________________________________________________________
batch_normalization_76 (Batc (None, 257, 2)            8         
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 255, 2)            14        
_________________________________________________________________
batch_normalization_77 (Batc (None, 255, 2)            8         
_________________________________________________________________
flatten_26 (Flatten)         (None, 510)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 1024)              523264    
_________________________________________________________________
reshape_26 (Reshape)         (None, 256, 4)            0         
__________

10000/10000 [==============================] - 3s 308us/step - loss: 0.5313 - acc: 0.8944 - val_loss: 0.4259 - val_acc: 0.9846
Epoch 3/3
10000/10000 [==============================] - 1s 142us/step
Test accuracy: [0.21727805149555207, 0.993878125]


In [0]:
# QAM-16
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(5,kernel_size=(3), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(5,kernel_size=(3)))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,bias_regularizer=regularizers.l2(0.005)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(BatchNormalization())
#modCNN.add(Dropout(0.4))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_49 (Conv1D)           (None, 255, 5)            35        
_________________________________________________________________
batch_normalization_73 (Batc (None, 255, 5)            20        
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 253, 5)            80        
_________________________________________________________________
batch_normalization_74 (Batc (None, 253, 5)            20        
_________________________________________________________________
flatten_25 (Flatten)         (None, 1265)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 4096)              5185536   
_________________________________________________________________
reshape_25 (Reshape)         (None, 256, 16)           0         
__________

50000/50000 [==============================] - 25s 507us/step - loss: 1.9520 - acc: 0.3473 - val_loss: 1.5289 - val_acc: 0.4553
Epoch 2/10
50000/50000 [==============================] - 21s 419us/step - loss: 1.3766 - acc: 0.5473 - val_loss: 1.2272 - val_acc: 0.5897
Epoch 3/10
 3712/50000 [=>............................] - ETA: 17s - loss: 1.1733 - acc: 0.6474

50000/50000 [==============================] - 21s 415us/step - loss: 1.1216 - acc: 0.6566 - val_loss: 1.0188 - val_acc: 0.6890
Epoch 4/10
50000/50000 [==============================] - 21s 415us/step - loss: 0.9363 - acc: 0.7375 - val_loss: 0.8583 - val_acc: 0.7649
Epoch 5/10
 4672/50000 [=>............................] - ETA: 17s - loss: 0.8156 - acc: 0.7925

50000/50000 [==============================] - 21s 416us/step - loss: 0.7880 - acc: 0.7997 - val_loss: 0.7222 - val_acc: 0.8215
Epoch 6/10
50000/50000 [==============================] - 21s 411us/step - loss: 0.6635 - acc: 0.8481 - val_loss: 0.6088 - val_acc: 0.8651
Epoch 7/10


50000/50000 [==============================] - 21s 411us/step - loss: 0.5584 - acc: 0.8847 - val_loss: 0.5116 - val_acc: 0.8981
Epoch 8/10
50000/50000 [==============================] - 20s 408us/step - loss: 0.4698 - acc: 0.9120 - val_loss: 0.4291 - val_acc: 0.9214
Epoch 9/10
 6912/50000 [===>..........................] - ETA: 16s - loss: 0.4136 - acc: 0.9261

50000/50000 [==============================] - 21s 411us/step - loss: 0.3962 - acc: 0.9320 - val_loss: 0.3629 - val_acc: 0.9393
Epoch 10/10
15136/50000 [========>.....................] - ETA: 5s

50000/50000 [==============================] - 8s 158us/step
Test accuracy: [0.308697285490036, 0.951788125]
